# RDF - Turtle - SPARQL Sandbox

In diesem Jupyter Notebook können "Hands-On" Erfahrungen mit RDF, Turtle und SPARQL gesammelt werden. Dabei kann nichts "kaputt" gehen...

## Nötige Module importieren

Das Ausführen der nächsten Zelle dauert beim ersten mal rund 1 Minute. Jede weitere Ausführung ist dann aber deutlich schneller...

In [ ]:
%pip install -q ipywidgets==8.1.1 ipycytoscape networkx rdflib
import rdflib
import ipycytoscape as cy
import networkx as nx
import pandas as pd
import requests
from IPython.display import HTML
import urllib3
from ext.rdf import parse_and_plot, local_query, remote_query, display_result

## Grundlagendokumente

Diese technischen Dokumente sind sehr lesenwert (und für Spezifikationen auch erstaunlich leserlich geschrieben):

- [RDF Primer](https://www.w3.org/TR/rdf11-primer/)
- [Turtle](https://www.w3.org/TR/turtle/)
- [SPARQL](https://www.w3.org/TR/sparql11-query/)

## Turtle

Turtle ist eine Möglichkeit, RDF zu serialisieren, also ein möglicher Syntax, um RDF Daten zu schreiben (und auch in einer Datenbank zu speichern). Um die Graph-Struktur hinter den RDF Daten zu sehen, werden diese jeweils direkt visualisiert, somit kann man auch gut erkennen, ob die Daten dem entsprechen, was beabsichtigt war.

### Erstes einfaches Turtle

In der nachfolgenden Zelle sind drei Triples, die einen einfachen Graphen bilden.

In [ ]:
ttl_string = """

<https://example.com/KingCharles> <https://example.com/hasChild> <https://example.com/PrinceHarry> .
<https://example.com/KingCharles> <https://example.com/isKingOf> <https://example.com/England> .
<https://example.com/KingCharles> <https://example.com/hasMother> <https://example.com/QueenElizabeth> .

"""

plot = parse_and_plot(ttl_string)
display(plot)

### Aufgabe 1

Schreibe in der nachfolgenden Zelle Triples für folgende Aussagen:

- Karin Keller-Sutter gehört dem Bundesrat an
- Karin Keller-Sutter ist in der FDP
- Karin Keller-Sutter wurde in Niederuzwil geboren

In [ ]:
ttl_string = """

<https://example.com/KarinKellerSutter> <https://example.com/mitgliedVon> <https://example.com/Bundesrat> .
<https://example.com/KarinKellerSutter> <https://example.com/inPartei> <https://example.com/FDP> .
<https://example.com/KarinKellerSutter> <https://example.com/geborenIn> <https://example.com/Niederuzwil> .

"""

plot = parse_and_plot(ttl_string)
display(plot)

### Prefix und Subjektwiederholung

Nachfolgende Zelle zeig zwei Vereinfachungen anhand der Informationen über King Charles:

- Nutzung eines Prefix `ex:` als Abkürzung für `https://example.com/`
- Auslassung des Subjekts, falls es wiederholt wird (Zeile vorher hört dann mit `;` auf statt `.`)

In [ ]:
ttl_string = """

@prefix ex: <https://example.com/> .

ex:KingCharles ex:hasChild ex:PrinceHarry ;
    ex:isKingOf ex:England ;
    ex:hasMother ex:QueenElizabeth .

"""

plot = parse_and_plot(ttl_string)
display(plot)

### Aufgabe 2

Schreibe die Triples aus Aufgabe 1 mit Prefix und wiederhole gleiche Subjekte nicht.

In [ ]:
ttl_string = """

@prefix ex: <https://example.com/> .

ex:KarinKellerSutter ex:mitgliedVon ex:Bundesrat ;
    ex:inPartei ex:FDP ;
    ex:geborenIn ex:Niederuzwil .

"""

plot = parse_and_plot(ttl_string)
display(plot)

### String Literals

Nachfolgende Zelle zeigt den Einsatz von String Literals:

- Zahlen können ohne `""` geschrieben werden
- Für Zeichenketten braucht es `""`
- Spezifische Datentypen können mit `^^URI` angehängt werden

In [ ]:
ttl_string = """

@prefix ex: <https://example.com/> .
@prefix xsd: <http://www.w3.org/2001/XMLSchema#> .

ex:KingCharles ex:hasChild ex:PrinceHarry ;
    ex:isKingOf ex:England ;
    ex:hasMother ex:QueenElizabeth ;
    ex:numberOfChildren 2 ;
    ex:fullName "Charles Philip Arthur George" ;
    ex:birthDate "1948-11-14"^^xsd:date .

"""

plot = parse_and_plot(ttl_string)
display(plot)

String Literals werden optisch anders dargestellt, als die URI Nodes.

### Aufgabe 3

Erweitere die Triples aus Aufgabe 2 um folgende Informationen

- Karin Keller-Sutter hat eine Beliebtheit von 4.56
- Sie trägt den vollen Namen "Karin Maria Keller-Sutter"
- Sie wurde am 22. Dezember 1963 geboren

In [ ]:
ttl_string = """

@prefix ex: <https://example.com/> .
@prefix xsd: <http://www.w3.org/2001/XMLSchema#> .

ex:KarinKellerSutter ex:mitgliedVon ex:Bundesrat ;
    ex:inPartei ex:FDP ;
    ex:geborenIn ex:Niederuzwil ;
    ex:beliebtheit 4.56 ;
    ex:vollerName "Karin Maria Keller-Sutter" ;
    ex:geburtsdatum "1963-12-22"^^xsd:date .

"""

plot = parse_and_plot(ttl_string)
display(plot)

### Weitere Subjekte

Nachfolgende Zelle zeigt neue Subjekte, die den Graphen etwas interessanter werden lassen:

In [ ]:
ttl_string = """

@prefix ex: <https://example.com/> .
@prefix xsd: <http://www.w3.org/2001/XMLSchema#> .

ex:KingCharles ex:hasChild ex:PrinceHarry ;
    ex:isKingOf ex:England ;
    ex:hasMother ex:QueenElizabeth ;
    ex:numberOfChildren 2 ;
    ex:fullName "Charles Philip Arthur George" ;
    ex:birthDate "1948-11-14"^^xsd:date .

ex:PrinceHarry ex:hasSibling ex:PrinceWilliam.
ex:QueenElizabeth ex:hasChild ex:KingCharles.
ex:PrinceWilliam ex:hasSibling ex:PrinceHarry;
    ex:livesIn ex:England.

"""

plot = parse_and_plot(ttl_string)
display(plot)

### Aufgabe 4

Integriere folgende Informationen in deinen Graphen über Karin Keller-Sutter:

- Der Name (String) der FDP ist "FDP.Die Liberalen"
- Niederuzwil ist Teil der Gemeinde Uzwil
- Uzwil liegt im Kanton St. Gallen
- Der Bundesrat ist die Exekutive der Schweiz
- Der Bundesrat hat als Mitglied Karin Keller-Sutter
- Die Schweiz hat Bern als Bundesstadt

In [ ]:
ttl_string = """

@prefix ex: <https://example.com/> .
@prefix xsd: <http://www.w3.org/2001/XMLSchema#> .

ex:KarinKellerSutter ex:mitgliedVon ex:Bundesrat ;
    ex:inPartei ex:FDP ;
    ex:geborenIn ex:Niederuzwil ;
    ex:beliebtheit 4.56 ;
    ex:vollerName "Karin Maria Keller-Sutter" ;
    ex:geburtsdatum "1963-12-22"^^xsd:date .

ex:FDP ex:name "FDP.Die Liberalen" .
ex:Niederuzwil ex:teilVon ex:Uzwil ;
    ex:inKanton ex:SG .
ex:Bundesrat ex:exekutiveVon ex:CH ;
    ex:hatMitglied ex:KarinKellerSutter .
ex:CH ex:hatBundesstadt ex:Bern .

"""

plot = parse_and_plot(ttl_string)
display(plot)

### Nodes zur Gruppierung von Information

Manchmal möchte man einer Aussage (einem Triple) noch weitere Metadaten anfügen, wie zum Beispiel von wann bis wann diese Aussage gestimmt hat, auf den ersten Blick ist das nicht ganz klar, wie hier vorzugehen ist. Wenn wir beispielsweise zu King Charles sagen wollen, dass er vom 14.11.1958 bis zum 7.9.2022 Kronprinz war und seit dem 8.9.2022 König, dann könnte man Folgendes probieren:

In [ ]:
ttl_string = """

@prefix ex: <https://example.com/> .
@prefix xsd: <http://www.w3.org/2001/XMLSchema#> .

ex:KingCharles ex:hasRole ex:CrownPrince ;
    ex:hasRole ex:King;
    ex:startedRole "1958-11-14"^^xsd:date;
    ex:endedRole "2022-09-07"^^xsd:date;
    ex:startedRole "2022-09-08"^^xsd:date.

"""

plot = parse_and_plot(ttl_string)
display(plot)

aber damit ist nicht klar, auf welche Rollen sich die Daten beziehen, eine andere Möglichkeit wäre:

In [ ]:
ttl_string = """

@prefix ex: <https://example.com/> .
@prefix xsd: <http://www.w3.org/2001/XMLSchema#> .

ex:KingCharles ex:hasRole ex:CrownPrince ;
    ex:hasRole ex:King .
ex:CrownPrince ex:begin "1958-11-14"^^xsd:date ;
    ex:end "2022-09-07"^^xsd:date .
ex:King ex:begin "2022-09-08"^^xsd:date .

"""

plot = parse_and_plot(ttl_string)
display(plot)

aber auch das wird zum Problem, spätestens wenn weitere Personen das Amt des Köngis oder Kronprinzen bekleiden, dann stimmen die Daten wieder nicht (ex:King ist ja einfach eine Entität König, da ist noch keine spezifische Person damit gemeint). Es braucht also eine andere Lösung...

### Aufgabe 5

Versuche folgende Informationen zu Karin Keller-Sutter in RDF zu modellieren:

- Sie war von 2011-2018 Ständerätin
- Sie ist seit 2019 Bundesrätin

In [ ]:
ttl_string = """

@prefix ex: <https://example.com/> .
@prefix xsd: <http://www.w3.org/2001/XMLSchema#> .

ex:KarinKellerSutter ex:hatRolle ex:Rolle1 .
ex:Rolle1 ex:beginn "2011"^^xsd:year ;
    ex:ende "2018"^^xsd:year ;
    ex:aufgabe ex:Ständerätin .
ex:KarinKellerSutter ex:hatRolle ex:Rolle2 .
ex:Rolle2 ex:beginn "2019"^^xsd:year ;
    ex:aufgabe ex:Bundesrätin .

"""

plot = parse_and_plot(ttl_string)
display(plot)

und für König Charles sähe es so aus:

In [ ]:
ttl_string = """

@prefix ex: <https://example.com/> .
@prefix xsd: <http://www.w3.org/2001/XMLSchema#> .

ex:KingCharles ex:hasRole ex:Role1 .
ex:Role1 ex:begin "1958-11-14"^^xsd:date ;
    ex:end "2022-09-07"^^xsd:date ;
    ex:function ex:CrownPrince .
ex:KingCharles ex:hasRole ex:Role2 .
ex:Role2 ex:begin "2022-09-08"^^xsd:date ;
    ex:function ex:King .

"""

plot = parse_and_plot(ttl_string)
display(plot)

### Blank Nodes

Diese Nodes zur Gruppierung interessieren ja nicht an und für sich, deshalb kann man sie auch durch sogenannte Blank Nodes ersetzen. Das sind Nodes, die keine eigene URI haben:

In [ ]:
ttl_string = """

@prefix ex: <https://example.com/> .
@prefix xsd: <http://www.w3.org/2001/XMLSchema#> .

ex:KingCharles ex:hasRole [
    ex:begin "1958-11-14"^^xsd:date ;
    ex:end "2022-09-07"^^xsd:date ;
    ex:function ex:CrownPrince ] .
ex:KingCharles ex:hasRole [
    ex:begin "2022-09-08"^^xsd:date ;
    ex:function ex:King ] .

"""

plot = parse_and_plot(ttl_string)
display(plot)

In der Visualisierung werden diese Nodes kleiner dargestellt und weisen keine URI auf.

## Nutzung von bestehenden Terms

### Aufgabe 6

Bilde nachfolgend mit Hilfe von [SKOS semantic relations terms](https://www.w3.org/TR/skos-reference/#semantic-relations) eine kleine Hierarchie bestehend aus folgenden Elementen:

- Holz
- Laubholz
- Nadelholz
- Birkenholz
- Tannenholz
- Buchenholz
- Laerchenholz

In [ ]:
ttl_string = """

@prefix ex: <https://example.com/> .
@prefix skos: <http://www.w3.org/2004/02/skos/core#> .

ex:Holz skos:narrower ex:Laubholz, ex:Nadelholz .
ex:Laubholz skos:narrower ex:Birkenholz, ex:Buchenholz .
ex:Nadelholz skos:narrower ex:Tannenholz, ex:Laerchenholz .

"""

plot = parse_and_plot(ttl_string)
display(plot)

### Aufgabe 7

Beschreibe Taylor Swift mit Terms von [schema.org](https://schema.org/Person):

In [ ]:
ttl_string = """

@prefix ex: <https://example.com/> .
@prefix schema: <https://schema.org/> .
@prefix xsd: <http://www.w3.org/2001/XMLSchema#> .

ex:TaylorSwift a schema:Person ;
    schema:birthDate "1989-12-13"^^xsd:date ;
    schema:birthPlace "Reading, Pennsylvania, USA" ;
    schema:description "Taylor Alison Swift is an American singer-songwriter." ;
    schema:award "Grammy Award for Album of the Year" ;
    schema:hasOccupation ex:Occupation1 .

ex:Occupation1 a schema:Occupation ;
    schema:name "Singer";
    schema:skill "Singing" .

ex:TaylorSwift schema:spouse schema:Spouse1, schema:Spouse2, schema:Spouse3, schema:Spouse4 .

schema:Spouse1 a schema:Person ;
    schema:name "Taylor Lautner" .
schema:Spouse2 a schema:Person ;
    schema:name "Calvin Harris" .
schema:Spouse3 a schema:Person ;
    schema:name "Joe Alvyn" .
schema:Spouse4 a schema:Person ;
    schema:name "Travis Kelce" .

"""

plot = parse_and_plot(ttl_string)
display(plot)

## SPARQL

Mit Hilfe von SPARQL Queries können RDF Daten abgefragt werden. Im Grundprinzip gibt man Query Patterns vor, die in den RDF Daten gesucht werden. Diese RDF Daten können von einem lokalen File kommen, oder von einem Triple Store, auf den über das Internet zugegriffen wird.

### Universale Query auf lokale Daten

Die nachfolgende Query ist die *universale Query*, die alle Triples in den Daten zurückgibt:

In [ ]:
ttl_string = """

@prefix ex: <https://example.com/> .
@prefix xsd: <http://www.w3.org/2001/XMLSchema#> .

ex:KingCharles ex:hasChild ex:PrinceHarry ;
    ex:isKingOf ex:England ;
    ex:hasMother ex:QueenElizabeth ;
    ex:numberOfChildren 2 ;
    ex:fullName "Charles Philip Arthur George" ;
    ex:birthDate "1948-11-14"^^xsd:date .

ex:PrinceHarry ex:hasSibling ex:PrinceWilliam.
ex:QueenElizabeth ex:hasChild ex:KingCharles.
ex:PrinceWilliam ex:hasSibling ex:PrinceHarry;
    ex:livesIn ex:England.

"""

query_string = """

SELECT ?s ?p ?o WHERE {

    ?s ?p ?o .

}

"""

df = local_query(ttl_string, query_string)
display_result(df)

### Abfrage nach bestimmtem Prädikat

Nachfolgende Query gibt alle Subjekte und Prädikate zurück, die über eine `ex:hasSibling` Verbindung verfügen:

In [ ]:
ttl_string = """

@prefix ex: <https://example.com/> .
@prefix xsd: <http://www.w3.org/2001/XMLSchema#> .

ex:KingCharles ex:hasChild ex:PrinceHarry ;
    ex:isKingOf ex:England ;
    ex:hasMother ex:QueenElizabeth ;
    ex:numberOfChildren 2 ;
    ex:fullName "Charles Philip Arthur George" ;
    ex:birthDate "1948-11-14"^^xsd:date .

ex:PrinceHarry ex:hasSibling ex:PrinceWilliam.
ex:QueenElizabeth ex:hasChild ex:KingCharles.
ex:PrinceWilliam ex:hasSibling ex:PrinceHarry;
    ex:livesIn ex:England.

"""

query_string = """

PREFIX ex: <https://example.com/>

SELECT ?s ?o WHERE {

    ?s ex:hasSibling ?o .

}

"""

df = local_query(ttl_string, query_string)
display_result(df)

### Remote Daten abfragen

Folgende Query gibt alle Kantone zurück, die bei Lindas gespeichert sind:

In [ ]:
query_string = """

PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>

SELECT * WHERE {

    ?canton rdf:type <https://schema.ld.admin.ch/Canton>  .

}

"""

df = remote_query(query_string, "https://ld.admin.ch/query")
display_result(df)

Da `rdf:type` ein so häufiges Prädikat ist, gibt es dafür die Abkürzung `a`:

In [ ]:
query_string = """

SELECT * WHERE {

    ?canton a <https://schema.ld.admin.ch/Canton>  .

}

"""

df = remote_query(query_string, "https://ld.admin.ch/query")
display_result(df)

### Aufgabe 8

Erstelle eine Query, die alle in der Gemeinde Payerne registrierten Unternehmungen mit ihrem Namen zurückgibt.

- Beispiel für eine Unternehmung: https://register.ld.admin.ch/zefix/company/3
- Beispiel für eine Gemeinde: https://ld.admin.ch/municipality/351

In [ ]:
query_string = """

PREFIX admin: <https://schema.ld.admin.ch/>
PREFIX schema: <http://schema.org/>

SELECT * WHERE {

    ?business a admin:ZefixOrganisation ;
        admin:municipality <https://ld.admin.ch/municipality/5822>;
        schema:name ?name .

}

"""

df = remote_query(query_string, "https://ld.admin.ch/query")
display_result(df)

## Shared Dimensions - Nutzung von bestehenden Objekten

### Alle Shared Dimensions von LINDAS

Die folgende Query listet alle Shared Dimensions von LINDAS und ihre deutschen Namen auf:

In [ ]:
query_string = """

PREFIX schema: <http://schema.org/>
PREFIX meta: <https://cube.link/meta/>

SELECT * WHERE {

    ?sd a meta:SharedDimension ;
        schema:name ?name .
  
  FILTER(lang(?name) = "de")

}

"""

df = remote_query(query_string, "https://ld.admin.ch/query")
display_result(df)

### Elemente in Shared Dimensions

Folgende Query listet alle Elemente (Objekte) der Shared Dimension "Parteien" (https://politics.ld.admin.ch/vocabulary/party) auf:

In [ ]:
query_string = """

PREFIX schema: <http://schema.org/>

SELECT * WHERE {

    <https://politics.ld.admin.ch/vocabulary/party> schema:hasDefinedTerm ?party.

    ?party schema:name ?name .
  
  FILTER(lang(?name) = "de")

}

"""

df = remote_query(query_string, "https://ld.admin.ch/query")
display_result(df)

### Aufgabe 9

Liste alle Elemente der Shared Dimension "Schwermetall" (https://ld.admin.ch/cube/dimension/el01) mit ihrem deutschen Namen auf. Hinweis, die Elemente dieser Shared Dimension sind nicht mit `schema:hasDefinedTerm` bei der Shared Dimension angehängt sondern auf Stufe Element mit `schema:inDefinedTermSet`:

In [ ]:
query_string = """

PREFIX schema: <http://schema.org/>

SELECT * WHERE {

    ?hm schema:inDefinedTermSet <https://ld.admin.ch/cube/dimension/el01> ;
      schema:name ?name .
  
  FILTER(lang(?name) = "de")

}

"""

df = remote_query(query_string, "https://ld.admin.ch/query")
display_result(df)